## Session 1

In [1]:
import numpy as np
import tensorflow as tf

/Users/simon/Programs/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# f: R^d -> R^c
def get_data(m, d, c):
    X = np.random.uniform(-1, +1, (m, d))
    A = np.random.uniform(-1, +1, (d, c))
    E = np.random.normal (0, 0.1, (m, c))
    Y = X.dot(A)+E
    
    return X, Y, A

In [4]:
X, Y, A = get_data(1000, 4, 2)
print(X.shape, Y.shape, A.shape)

((1000, 4), (1000, 2), (4, 2))


In [5]:
print(A)

[[-0.81302658  0.28554027]
 [ 0.26367177  0.05640826]
 [-0.44034922 -0.48801552]
 [-0.75912434 -0.58244918]]


# The idea is to recover A

In [6]:
X_tf = tf.constant(X, dtype=tf.float32) #Tensor flow variant of X
Y_tf = tf.constant(Y, dtype=tf.float32)
A_tf = tf.Variable(np.zeros(A.shape), dtype=tf.float32) #be careful with the capital V
F_tf = tf.matmul(X_tf, A_tf) # Define the mapping between X->A

loss_tf = tf.reduce_mean(tf.square(F_tf-Y_tf)) #How good is the F prediction vs the real Y, be careful that the data type should be the same

#Definition of the optimizer, specify the tensor you want to minimize
step_tf = tf.train.GradientDescentOptimizer(1E-3).minimize(loss_tf) 


#We need a session, this is just syntaxis

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer()) #Gives you a vertex with a function to initialise all variables in the graph
    
    num_iterations, print_every = 2**14, 2**10
    for i in range(num_iterations):
        
        sess.run(step_tf)
        
        if i % print_every == print_every-1:
            print(sess.run(loss_tf))   #The loss is a fucntion that has to be executed as well
            #When we dont use sess.run it just defines a vertex on the graph
    
        
    A_opt = sess.run(A_tf) #We excecute the optimal A up to that point


0.19000438
0.102641314
0.05766635
0.0344423
0.022414394
0.016167065
0.012913189
0.011213923
0.010324249
0.009857313
0.009611679
0.00948218
0.009413769
0.009377553
0.009358351
0.00934815


# They are not the same cus we added noise on top of the original Y = X.dot(A)+E, E being the gaussian noise 

In [7]:
print(A)
print("")
print(A_opt)

[[-0.81302658  0.28554027]
 [ 0.26367177  0.05640826]
 [-0.44034922 -0.48801552]
 [-0.75912434 -0.58244918]]

[[-0.8155286   0.28901714]
 [ 0.26406878  0.05772051]
 [-0.43799645 -0.4838686 ]
 [-0.7520337  -0.5815256 ]]
